<a href="https://colab.research.google.com/github/surajsy/ColabNotebooks/blob/master/Neuro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.version

'3.6.9 (default, Apr 18 2020, 01:56:04) \n[GCC 8.4.0]'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

import scipy.io as sio
import glob
import numpy as np

In [ ]:
path = '/content/gdrive/My Drive/PRNN/Neuro_dataset/'

alzh_aal_data = []
alzh_power_data = []
for f in glob.iglob(path+'Alzheimer*/*',recursive=True):
  content = sio.loadmat(f)
  alzh_aal_data.append(content['tc_rest_aal'].astype(np.float32))
  alzh_power_data.append(content['tc_rest_power'].astype(np.float32))

normal_aal_data = []
normal_power_data = []
for f in glob.iglob(path+'Normal*/*',recursive=True):
  content = sio.loadmat(f)
  normal_aal_data.append(content['tc_rest_aal'].astype(np.float32))
  normal_power_data.append(content['tc_rest_power'].astype(np.float32))

all_aal_data = alzh_aal_data + normal_aal_data
all_power_data = alzh_power_data + normal_power_data
print(np.shape(all_aal_data))
print(np.shape(all_power_data))

(81, 140, 116)
(81, 140, 264)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = StandardScaler()
all_aal_data = list(map(lambda X: scaler.fit_transform(X), all_aal_data))
all_power_data = list(map(lambda X: scaler.fit_transform(X), all_power_data))

In [ ]:
labels = list(
    np.concatenate(
        (np.zeros(len(alzh_aal_data), dtype=np.long),
          np.ones(len(normal_aal_data), dtype=np.long))
        )
    )
len(labels)

81

In [ ]:
def train_fnn(train_loader, net, num_epochs, reshape):
  for epoch in range(num_epochs):
      for i, (data, labels) in enumerate(train_loader):
        data = Variable(data.view(reshape))
        # print(np.shape(data))         
        labels = Variable(labels)
          
        if use_cuda and torch.cuda.is_available():
            data = data.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             
        outputs = net(data)
        # print(np.shape(outputs.squeeze(axis=1)), np.shape(labels))
        # print(np.shape(outputs))                            
        loss = criterion(outputs.squeeze(axis=1), labels)                 
        loss.backward()                                   
        optimizer.step()                                  
        
        # if (i+1) % 10 == 0:                             
        #     print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
        #         %(epoch+1, num_epochs, i+1, len(data)//batch_size, loss.item()))

def test_fnn(test_loader, net, reshape):              
  correct = 0
  total = 0
  for data, labels in test_loader:
    data = Variable(data.view(reshape))
    
    if use_cuda and torch.cuda.is_available():
        data = data.cuda()
        labels = labels.cuda()
    
    
    outputs = net(data)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)                    
    correct += (predicted == labels).sum()     
      
  print('Accuracy of the network on the 30 test samples: %d %%' % (100 * correct / total))

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size[0]) 
        self.relu = nn.Tanh()                          
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.elu = nn.Tanh()
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        self.tanh = nn.Tanh()
        self.fc4 = nn.Linear(hidden_size[2],hidden_size[3])
        self.relu1 = nn.Tanh()
        self.fc5 = nn.Linear(hidden_size[3], num_classes)
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.elu(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.relu1(out)
        out = self.fc5(out)
        return out

In [ ]:
class LSTMnet(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTMnet, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.dropout = nn.Dropout(p=0.1)
    self.fc1 = nn.Linear(hidden_size, 128)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self, x):
    _, (out, cn) = self.lstm(x)
    out = self.dropout(out)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [ ]:
class LSTMnet2(nn.Module):
  def __init__(self, input_size1, input_size2, hidden_size1, hidden_size2, num_layers, num_classes):
    super(LSTMnet2, self).__init__()
    self.lstm1 = nn.LSTM(input_size1, hidden_size1, num_layers, batch_first=True)
    self.lstm2 = nn.LSTM(input_size2, hidden_size2, num_layers, batch_first=True)
    self.dropout = nn.Dropout(p=0.1)
    self.fc1 = nn.Linear(hidden_size1+hidden_size2, 128)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self, x, y):
    _, (out1, cn) = self.lstm1(x)
    _, (out2, cn) = self.lstm2(y)
    # print(np.shape(out1), np.shape(out2))
    out = torch.cat((out1, out2), dim=2)
    out = self.dropout(out)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [ ]:
train, test = torch.utils.data.random_split(list(zip(all_aal_data, labels)), [54, 27])

seq_len       = 140
input_size    = 116  
hidden_size   = [2048, 2048, 2048, 2048]
num_classes   = 2
num_epochs    = 25     
batch_size    = 1   
learning_rate = 1e-3

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

net = Net(input_size*seq_len, hidden_size, num_classes)
use_cuda = True

if use_cuda and torch.cuda.is_available():
    net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

train_fnn(train_loader, net, num_epochs, [batch_size, input_size*seq_len])
test_fnn(test_loader, net, [batch_size, input_size*seq_len])

Accuracy of the network on the 30 test samples: 40 %


In [ ]:
train, test = torch.utils.data.random_split(list(zip(all_aal_data, labels)), [54, 27])

input_size    = 116
seq_len       = 140  
hidden_size   = 70
num_layers = 1
num_classes   = 2
num_epochs    = 25     
batch_size    = 1   
learning_rate = 1e-3

lstm_net = LSTMnet(input_size, hidden_size, num_layers, num_classes)

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

use_cuda = True

if use_cuda and torch.cuda.is_available():
    lstm_net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_net.parameters(), lr=learning_rate)

train_fnn(train_loader, lstm_net, num_epochs, [batch_size, seq_len, input_size])
test_fnn(test_loader, lstm_net, [batch_size, seq_len, input_size])

Accuracy of the network on the 30 test samples: 66 %


In [ ]:
train, test = torch.utils.data.random_split(list(zip(all_power_data, labels)), [54, 27])

seq_len       = 140
input_size    = 264  
hidden_size   = [2048, 512, 256, 64]
num_classes   = 2
num_epochs    = 25     
batch_size    = 1   
learning_rate = 1e-3

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

net = Net(input_size*seq_len, hidden_size, num_classes)

use_cuda = True

if use_cuda and torch.cuda.is_available():
    net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

train_fnn(train_loader, net, num_epochs, [batch_size, input_size*seq_len])
test_fnn(test_loader, net, [batch_size, seq_len*input_size])

Accuracy of the network on the 30 test samples: 55 %


In [ ]:
train, test = torch.utils.data.random_split(list(zip(all_power_data, labels)), [54, 27])

input_size    = 264
seq_len       = 140  
hidden_size   = 132
num_layers = 1
num_classes   = 2
num_epochs    = 25    
batch_size    = 1   
learning_rate = 1e-3

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=True)

lstm_net = LSTMnet(input_size, hidden_size, num_layers, num_classes)

use_cuda = True

if use_cuda and torch.cuda.is_available():
    lstm_net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_net.parameters(), lr=learning_rate)

train_fnn(train_loader, lstm_net, num_epochs, [batch_size, seq_len, input_size])
test_fnn(test_loader, lstm_net, [batch_size, seq_len, input_size])

Accuracy of the network on the 30 test samples: 81 %


In [ ]:
train, test = torch.utils.data.random_split(list(zip(all_aal_data, all_power_data, labels)), [54, 27])

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

input_size1 = 116
input_size2   = 264
seq_len       = 140
hidden_size1 = 58  
hidden_size2   = 132
num_layers = 1
num_classes   = 2
num_epochs    = 25    
batch_size    = 1   
learning_rate = 1e-3

lstm_net2 = LSTMnet2(input_size1, input_size2, hidden_size1, hidden_size2, num_layers, num_classes)

use_cuda = True

if use_cuda and torch.cuda.is_available():
    lstm_net2.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_net.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i, (data1, data2, label) in enumerate(train_loader):
      data1 = Variable(data1.view(-1, seq_len, input_size1))
      data2 = Variable(data2.view(-1, seq_len, input_size2))
      # print(np.shape(data))         
      label = Variable(label)
        
      if use_cuda and torch.cuda.is_available():
          data1 = data1.cuda()
          data2 = data2.cuda()
          label = label.cuda()
      
      optimizer.zero_grad()                             
      outputs = lstm_net2(data1, data2)
      # print(np.shape(outputs.squeeze(axis=1)), np.shape(labels))
      # print(np.shape(outputs))                            
      loss = criterion(outputs.squeeze(axis=1), label)                 
      loss.backward()                                   
      optimizer.step()                                  
      
      # if (i+1) % 10 == 0:                             
      #     print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
              # %(epoch+1, num_epochs, i+1, len(data1)//batch_size, loss.item()))

In [ ]:
  correct = 0
  total = 0
  for data1, data2, label in test_loader:
    data1 = Variable(data1.view(-1, seq_len, input_size1))
    data2 = Variable(data2.view(-1, seq_len, input_size2))
    
    if use_cuda and torch.cuda.is_available():
        data1 = data1.cuda()
        data2 = data2.cuda()
        label = label.cuda()
    
    
    outputs = lstm_net2(data1, data2)
    _, predicted = torch.max(outputs.data, 1)
    total += label.size(0)                    
    correct += (predicted == label).sum()     
      
  print('Accuracy of the network on the 30 test samples: %d %%' % (100 * correct / total))

Accuracy of the network on the 30 test samples: 88 %


In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
flat_aal_data = np.reshape(np.array(all_aal_data),(len(all_aal_data), -1))
flat_power_data = np.reshape(np.array(all_power_data),(len(all_power_data), -1))
flat_data = np.concatenate((flat_aal_data, flat_power_data), axis=1)

In [ ]:
kernel = ['rbf']
C = [0.0001, 0.1, 1, 10]

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(flat_aal_data, labels,test_size=0.3,shuffle=True)

for c in C:
  for k in kernel:
    clf = SVC(C=c, kernel=k)

    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    print('kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(test_y,pred))

kernel: rbf 	C: 0.0001 	SVM Accuracy: 68.0
kernel: rbf 	C: 0.1 	SVM Accuracy: 68.0
kernel: rbf 	C: 1 	SVM Accuracy: 68.0
kernel: rbf 	C: 10 	SVM Accuracy: 44.0


In [ ]:
train_X, test_X, train_y, test_y = train_test_split(flat_power_data, labels,test_size=0.3,shuffle=True)

for c in C:
  for k in kernel:
    clf = SVC(C=c, kernel=k)

    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    print('kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(test_y,pred))

kernel: rbf 	C: 0.0001 	SVM Accuracy: 28.000000000000004
kernel: rbf 	C: 0.1 	SVM Accuracy: 28.000000000000004
kernel: rbf 	C: 1 	SVM Accuracy: 36.0
kernel: rbf 	C: 10 	SVM Accuracy: 52.0


In [ ]:
train_X, test_X, train_y, test_y = train_test_split(flat_data, labels,test_size=0.3,shuffle=True)

for c in C:
  for k in kernel:
    clf = SVC(C=c, kernel=k)

    clf.fit(train_X, train_y)
    pred = clf.predict(test_X)
    print('kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(test_y,pred))

kernel: rbf 	C: 0.0001 	SVM Accuracy: 56.00000000000001
kernel: rbf 	C: 0.1 	SVM Accuracy: 56.00000000000001
kernel: rbf 	C: 1 	SVM Accuracy: 56.00000000000001
kernel: rbf 	C: 10 	SVM Accuracy: 60.0


In [ ]:
np.shape(np.array(all_power_data[0]).T)

(264, 140)